In [1]:
import os, glob
import json

import numpy as np
import pandas as pd


In [40]:
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

lookup["urn:cts:greekLit:tlg0640.tlg001"]

['tlg0640.tlg001.ogl-eng1.xml', 'alciphron_1896']

In [41]:
for path in ["/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1.xml"]:
    prefix = os.path.splitext(path)[0]
    print(prefix)

/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1


In [74]:
text_ctsurn = []
num_transl = []
transl_langs = []
num_src_sents = []
text_top_1s = []
text_top_5s = []
text_top_10s = []
num_tgt_sents = []
num_nulls_all_transl = []
names_translations = []
transl_top10s = []
source_edition = []

not_transl_keys = ["num_src_sents", "num_translations", "text_top1", "text_top5", "text_top10"]

eval_dicts_dir = "/home/craig.car/repos/chiron/chironata/eval_datafiles/"
for eval_json_path in glob.iglob(eval_dicts_dir+"*.json"):
    ctsurn = os.path.splitext(os.path.basename(eval_json_path))[0]
    with open(eval_json_path) as f:
        eval_json = json.load(f)
    text_ctsurn.append(ctsurn)
    # print(eval_json["apolloniusRhodius_1_1791"].keys())
    num_transl.append(eval_json["num_translations"])
    num_src_sents.append(eval_json["num_src_sents"])
    text_top_1s.append(eval_json["text_top1"])
    text_top_5s.append(eval_json["text_top5"])
    text_top_10s.append(eval_json["text_top10"])
    text_langs = set()
    num_transl_sents = 0
    num_nulls = []
    transl_names = []
    top10s_transl = []
    translations = []
    for key in eval_json.keys():
        if key not in not_transl_keys:
            # print(eval_json[key].keys())
            text_langs.add(eval_json[key]["tgt_lang"])
            num_transl_sents += eval_json[key]["num_tgt_sents"]
            transl_names.append(key)
            top10s_transl.append("%.3f" % eval_json[key]["transl_top10"])
            num_nulls.append(eval_json[key]["num_nulls"])
            # print(num_transl_sents)
    transl_langs.append(text_langs)
    num_tgt_sents.append(num_transl_sents)
    names_translations.append(transl_names)
    transl_top10s.append(top10s_transl)
    num_nulls_all_transl.append(num_nulls)


In [75]:
print(len(text_ctsurn))
print(len(num_transl))
print(len(transl_langs))
print(len(num_src_sents))
print(len(text_top_1s))
print(len(text_top_5s))
print(len(text_top_10s))
print(len(num_tgt_sents))
print(len(num_nulls_all_transl))
print(len(names_translations))
print(len(transl_top10s))
print(len(num_nulls_all_transl))

709
709
709
709
709
709
709
709
709
709
709
709


In [76]:
eval_df = pd.DataFrame(text_ctsurn)
eval_df.columns = ["ctsurn"]

In [77]:
eval_df["number_src_sents"] = num_src_sents
eval_df["number_translations"] = num_transl
eval_df["names_translations"] = names_translations
eval_df["translation_languages"] = transl_langs
eval_df["num_tgt_sents_all_translations"] = num_tgt_sents
eval_df["num_src2null_all_translations"] = num_nulls_all_transl
eval_df["text_wide_recall@1"] = text_top_1s
eval_df["text_wide_recall@5"] = text_top_5s
eval_df["text_wide_recall@10"] = text_top_10s
eval_df["translations_recall@10"] = transl_top10s

In [78]:
eval_df.round(3)

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
0,urn:cts:greekLit:tlg0001.tlg001,1738,3,"[apolloniusRhodius_1_1791, tlg0001.tlg001.opp-...","{fr, en, it}",18327,"[0, 0, 0]",0.179,0.296,0.353,"[0.024, 0.605, 0.432]"
1,urn:cts:greekLit:tlg0003.tlg001,4419,7,"[thucydides_1_1864, thucydides_1_1869, thucydi...","{de, en, fr}",53054,"[0, 1, 173, 36, 275, 62, 0]",0.130,0.194,0.225,"[0.698, 0.659, 0.010, 0.156, 0.010, 0.010, 0.032]"
2,urn:cts:greekLit:tlg0004.tlg001,6440,1,[diogenesLaertius_1853],{en},11907,[3],0.436,0.654,0.737,[0.737]
3,urn:cts:greekLit:tlg0005.tlg001,549,5,"[theocritus_1855, theocritus_1878, theocritus_...","{de, fr, en}",23922,"[1, 0, 0, 0, 0]",0.234,0.409,0.499,"[0.468, 0.605, 0.576, 0.470, 0.377]"
4,urn:cts:greekLit:tlg0005.tlg002,94,5,"[theocritus_1855, theocritus_1878, theocritus_...","{de, fr, en}",23922,"[0, 0, 0, 0, 0]",0.123,0.200,0.262,"[0.298, 0.447, 0.181, 0.234, 0.149]"
...,...,...,...,...,...,...,...,...,...,...,...
704,urn:cts:latinLit:stoa0255.stoa010,1539,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.664,0.858,0.919,"[0.905, 0.933]"
705,urn:cts:latinLit:stoa0255.stoa011,110,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.686,0.855,0.914,"[0.855, 0.973]"
706,urn:cts:latinLit:stoa0255.stoa012,295,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.646,0.814,0.863,"[0.769, 0.956]"
707,urn:cts:latinLit:stoa0255.stoa013,460,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.687,0.872,0.936,"[0.915, 0.957]"


In [79]:
# get corpus-level stats
corpus_top1 = eval_df["text_wide_recall@1"].sum()/eval_df.shape[0]
corpus_top5 = eval_df["text_wide_recall@5"].sum()/eval_df.shape[0]
corpus_top10 = eval_df["text_wide_recall@10"].sum()/eval_df.shape[0]

print(corpus_top1)
print(corpus_top5)
print(corpus_top10)

0.21667578986224637
0.3270881878940083
0.3876063934476651


In [85]:
eval_df.sort_values("text_wide_recall@10", ascending=[False]).head(25)

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
661,urn:cts:latinLit:phi1212.phi003,392,1,[apuleius_1878],{en},8847,[0],0.757653,0.989796,1.043367,[1.043]
595,urn:cts:latinLit:phi0474.phi055,2011,1,[cicero_1874],{en},5692,[0],0.776231,0.945301,0.987568,[0.988]
666,urn:cts:latinLit:phi1348.abo011,517,2,"[suetonius_1855-1897, caesar_commentaries_1867]","{de, fr}",13468,"[0, 0]",0.709865,0.913926,0.984526,"[0.971, 0.998]"
433,urn:cts:greekLit:tlg0074.tlg004,282,1,[arrian_1860],{fr},3216,[0],0.549645,0.872340,0.971631,[0.972]
323,urn:cts:greekLit:tlg0034.tlg001,406,1,[lycurgus_1869],{de},655,[0],0.655172,0.874384,0.963054,[0.963]
659,urn:cts:latinLit:phi1212.phi001,1314,1,[apuleius_1878],{en},8847,[0],0.705479,0.898021,0.953577,[0.954]
615,urn:cts:latinLit:phi0588.abo015,89,2,"[justin_cNepos_eutropius_1902, corneliusNepos_...","{de, en}",12161,"[0, 0]",0.735955,0.870787,0.938202,"[0.933, 0.944]"
707,urn:cts:latinLit:stoa0255.stoa013,460,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.686957,0.871739,0.935870,"[0.915, 0.957]"
708,urn:cts:latinLit:stoa0255.stoa014,435,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.673563,0.859770,0.931034,"[0.961, 0.901]"
699,urn:cts:latinLit:stoa0255.stoa004,387,2,"[seneca_1877, seneca_1889]","{en, fr}",36068,"[0, 0]",0.670543,0.863049,0.928941,"[0.904, 0.953]"


In [86]:
np.average(np.array(eval_df.sort_values("text_wide_recall@10", ascending=[False])["number_src_sents"]))

817.6459802538787

In [87]:
eval_df["number_src_sents"].sum()

579711

In [88]:
eval_df["num_tgt_sents_all_translations"].sum()

16430361

In [89]:
eval_df["number_translations"].sum()

2102

In [90]:
eval_df.sort_values("text_wide_recall@10", ascending=[True]).head(25)

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
434,urn:cts:greekLit:tlg0074.tlg008,9,1,[epictetus_1862],{fr},9106,[0],0.000000,0.000000,0.000000,[0.000]
582,urn:cts:latinLit:phi0474.phi035,4047,2,"[cicero_1855-1892_11, cicero_1868_12]",{de},9216,"[8, 2]",0.000741,0.005313,0.008772,"[0.010, 0.008]"
596,urn:cts:latinLit:phi0474.phi056,6913,2,"[cicero_1855-1897_2, cicero_1855-1903_1]",{de},11366,"[398, 634]",0.001374,0.004484,0.008969,"[0.010, 0.008]"
463,urn:cts:greekLit:tlg0096.tlg002,6062,1,[aesopHesiodQuintus_1865-1866],{de},7368,[1],0.000660,0.006434,0.012042,[0.012]
597,urn:cts:latinLit:phi0474.phi057,10677,3,"[cicero_1874, cicero_1855-1897_2, cicero_1855-...","{de, en}",17058,"[2629, 2303, 2432]",0.002716,0.007586,0.012394,"[0.010, 0.017, 0.011]"
515,urn:cts:greekLit:tlg0551.tlg017,3744,3,"[appian_1832_10, Appianus_1828, appian_1830_5]",{de},26318,"[0, 4, 0]",0.001781,0.007746,0.013177,"[0.014, 0.015, 0.010]"
521,urn:cts:greekLit:tlg0563.tlg001,2288,1,[sophistesGrecs_1842],{fr},7988,[0],0.000437,0.003934,0.013986,[0.014]
698,urn:cts:latinLit:stoa0215c.stoa001,4074,1,[alfredGreat_orosius_1873],{en},5828,[4],0.004418,0.009818,0.016446,[0.016]
532,urn:cts:greekLit:tlg2034.tlg007,2591,2,"[aristotle_1_1908, aristotle_2_1902]",{en},12615,"[2, 0]",0.002316,0.011772,0.020069,"[0.018, 0.022]"
343,urn:cts:greekLit:tlg0059.tlg013,2013,2,"[plato_3_1826, plato_1_1846]",{fr},7106,"[0, 0]",0.001739,0.010432,0.020368,"[0.017, 0.023]"


In [91]:
np.average(np.array(eval_df.sort_values("text_wide_recall@10", ascending=[True])["number_src_sents"]))

817.6459802538787

In [92]:
eval_df.sort_values("num_src2null_all_translations")

,ctsurn,number_src_sents,number_translations,names_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10,translations_recall@10
527,urn:cts:greekLit:tlg1247.tlg001,375,1,[Epictetus_MarcusAurelius_Theophrastus_1855-1891],{de},5160,[0],0.021333,0.061333,0.128000,[0.128]
475,urn:cts:greekLit:tlg0540.tlg010,72,1,[lysias_1856],{de},5319,[0],0.222222,0.444444,0.472222,[0.472]
474,urn:cts:greekLit:tlg0540.tlg009,37,1,[lysias_1856],{de},5319,[0],0.054054,0.162162,0.216216,[0.216]
473,urn:cts:greekLit:tlg0540.tlg008,50,1,[lysias_1856],{de},5319,[0],0.380000,0.640000,0.720000,[0.720]
472,urn:cts:greekLit:tlg0540.tlg007,66,1,[lysias_1856],{de},5319,[0],0.318182,0.424242,0.545455,[0.545]
...,...,...,...,...,...,...,...,...,...,...,...
435,urn:cts:greekLit:tlg0081.tlg001,7516,2,"[dionysiusHalicarnassus_2_1738, dionysiusHalic...",{it},8341,"[3438, 3369]",0.206559,0.273417,0.301823,"[0.267, 0.336]"
531,urn:cts:greekLit:tlg2000.tlg001,12496,3,"[plotinus_2_1905, plotinus_1895, plotinus_1_1905]","{de, en}",14361,"[6870, 4202, 6314]",0.029583,0.049536,0.058552,"[0.073, 0.060, 0.043]"
682,urn:cts:latinLit:stoa0040.stoa003,17454,1,[Augustine-cityofgodtransla02auguuoft],{en},9020,[7195],0.301421,0.389596,0.418586,[0.419]
652,urn:cts:latinLit:phi0978.phi001,29284,12,"[PlinyYounger_1855, plinyYounger_1854_14, Plin...","{de, fr}",101002,"[13803, 15199, 14174, 10783, 15335, 1377, 11432]",0.027669,0.038730,0.043573,"[0.029, 0.089, 0.107, 0.238, 0.009, 0.045, 0.006]"


In [93]:
eval_df[["text_wide_recall@1", "text_wide_recall@5", "text_wide_recall@10"]].describe()

,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10
count,709.000000,709.000000,709.000000
mean,0.216676,0.327088,0.387606
std,0.169975,0.204115,0.211675
min,0.000000,0.000000,0.000000
25%,0.085965,0.180952,0.246753
50%,0.180055,0.296970,0.363636
75%,0.306818,0.449923,0.511438
max,0.776231,0.989796,1.043367
